In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Students/Negin/

/content/drive/My Drive/Colab Notebooks/Students/Negin


In [21]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')
from util import eval, loadDataset, normalization, kmeansInitialization

eps = torch.tensor(10 ** -10)

datasets = ['3Sources.npy',
'BBCSport.npy',
'Caltech101.npy',
'Caltech_2.npy',
'Citeseer.npy',
'Coil100.npy',
'Cora.npy',
'EYaleB10.npy',
'Handwritten.npy',
'MNIST10.npy',
'UCIdigit.npy',
'Umist.npy',
'Yale32.npy',
'Yeast.npy',
'Cora2.npy',
'texas.npy',
'wisconsin.npy',
'washington.npy',
'cornell.npy']


X0, Y, V, c = loadDataset(10)


r = 40
k = 5

rr = 2

sigma = .1
beta = .1
gamma = .1
lamda = .1
theta = .1

iter = 50

W = []
H = []
X = []

O = torch.ones(r, r)

for v in range(V):


    X.append(torch.tensor(X0[v]).type(torch.float32))

    d, n = X[v].shape

    W.append(torch.rand(d, r))

H = torch.rand(r, n)
C = torch.rand(r, c)
S = (1 - theta) * torch.eye(c) + theta * torch.ones(c, c)
G = torch.rand(c, n)

An = []
Ad = []
Dn = []
Dd = []
for v in range(V):

    An0 = kneighbors_graph(X[v].T, k, mode='connectivity', include_self=False).toarray()
    An0 = torch.tensor(An0).type(torch.float32)
    An0 = torch.maximum(An0, An0.T)

    An.append(An0)

    Dn0 = torch.diag(torch.sum(An0, dim = 1))
    Dn.append(Dn0)


    Ad0 = kneighbors_graph(X[v], k, mode='connectivity', include_self=False).toarray()
    Ad0 = torch.tensor(Ad0).type(torch.float32)
    Ad0 = torch.maximum(Ad0, Ad0.T)

    Ad.append(Ad0)

    Dd0 = torch.diag(torch.sum(Ad0, dim = 1))
    Dd.append(Dd0)

alpha = torch.ones(V)/V
# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(V):

        # Updating Ws
        Wn = (alpha[v] ** rr) * (X[v]     @ H.T) + lamda * (Ad[v] @ W[v]) + sigma * W[v]
        Wd = (alpha[v] ** rr) * (W[v] @ H @ H.T) + lamda * (Dd[v] @ W[v]) + sigma * (W[v] @ W[v].T @ W[v])
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps))

    # Updating H

    Hp = C @ S @ G
    Hp[Hp<0] = 0
    Hn = (alpha[v] ** rr) * (W[v].T @ X[v])     + beta * Hp + gamma * (H @ An[v])
    Hd = (alpha[v] ** rr) * (W[v].T @ W[v] @ H) + beta * H  + gamma * (H @ Dn[v])
    H = H * (Hn / torch.maximum(Hd, eps))

    # Updating C
    # C = H @ G.T @ S.T @ torch.inverse(S @ G @ G.T @ S.T)
    C = H @ G.T @ S.T @ (1/torch.maximum(S @ G @ G.T @ S.T, eps))


    # Updating G
    dis = torch.cdist(H.T, S.T @ C.T)
    mem = torch.argmin(dis, dim=1)
    G = (F.one_hot(mem, num_classes=c).T).type(torch.float32)

    # Updating Alphas
    for v in range(V):
        alpha[v] = (torch.norm(X[v] - W[v] @ H) **2) ** (1/(1 - rr))
    alpha = alpha / torch.sum(alpha)
        # Calculating cost function
#     err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + gamma * torch.trace(H[v] @ (D[v] - A[v]) @ H[v].T)

# plt.plot(err)


# Hsum = torch.zeros(r, n)
# for v in range(V):
#     Hsum += H[v]

# Hfinal = Hsum / V

# pred = KMeans(n_clusters=c).fit(G.T).labels_

pred = torch.argmax(G,dim=0)

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

(0.0,
 0.1,
 0.0,
 0.10000000000000002,
 0.01818181818181818,
 0.018181818181818184,
 0.1)

In [20]:
W

[tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0029],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0022],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0024, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0106, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0059, 0.0000]]),
 tensor([[0.0130, 0.0042, 0.0022,  ..., 0.0015, 0.0201, 0.0065],
         [0.0035, 0.0321, 0.0116,  ..., 0.0066, 0.0191, 0.0185],
         [0.0182, 0.0159, 0.0269,  ..., 0.0163, 0.0365, 0.0048],
         ...,
         [0.0033, 0.0365, 0.0094,  ..., 0.0203, 0.0041, 0.0107],
         [0.0061, 0.0084, 0.0133,  ..., 0.0092, 0.0122, 0.0061],
         [0.0038, 0.0311, 0.0039,  ..., 0.0414, 0.0023, 0.0065]]),
 tensor([[0.0233, 0.0174, 0.0199,  ..., 0.0229, 0.0185, 0.0200],
         [0.0233, 0.0174, 0.0199,  ..., 0.0229, 0.0185, 0.0200],
         [0.0233, 0.0174, 0.0199,  ..., 0.0229, 0.0185, 0.